In [1]:
import numpy as np
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing
import tensorflow as tf
import numpy as np
import tvm.relay as relay
from tvm import relay
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *

import json
import numpy as np
import pygraphviz as pgv

def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
            #A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(shape_size(json_data['attrs']['shape'][1][src[0]])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(shape_size(json_data['attrs']['shape'][1][node_idx])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

2022-04-12 11:25:47.317208: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [15]:
model_keras = tf.keras.models.load_model('./very_simple_model.h5')

input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

In [ ]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

executor = graph_executor.GraphModule(lib['default'](dev))


In [16]:
print(mod)

def @main(%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %6 = nn.conv2d(%5, %v_param_5, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]);
  %7 = nn.bias_add(%6, %v_param_6);
  nn.max_pool2d(%7, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
}



In [14]:
with relay.quantize.qconfig(calibrate_mode="global_scale", global_scale=8.0):
    mod = relay.quantize.quantize(mod, params)
print(mod)

def @main(%input_1: Tensor[(1, 3, 256, 256), float32]) -> Tensor[(1, 64, 32, 32), float32] {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %1 = annotation.stop_fusion(%0) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %2 = nn.bias_add(%1, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %3 = nn.max_pool2d(%2, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %4 = multiply(%3, 16f /* ty=float32 */) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %5 = round(%4) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %6 = clip(%5, a_min=-127f, a_max=127f) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %7 = cast(%6, dtype="int8") /* ty=Tensor[(1, 16, 128, 128), int8] */;
  %8 = nn.conv2d(%7, meta[relay.Constant][2] /* ty=Tensor[(32, 16, 3, 3), int8]

In [3]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("unet", 512))

input_data = np.random.normal(0,1,(1,512,512,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

2022-04-11 16:04:23.049023: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-11 16:04:23.049084: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-11 16:04:23.080980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 16:04:23.081265: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-11 16:04:23.081281: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-11 16:04:23.082606: I tensorflow/stream_executor/platfor

In [2]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("resnet152", 224))

input_data = np.random.normal(0,1,(1,224,224,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

2022-04-12 11:26:08.526300: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-12 11:26:08.526379: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-12 11:26:08.557157: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 11:26:08.557440: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-12 11:26:08.557454: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-12 11:26:08.558850: I tensorflow/stream_executor/platfor

In [ ]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        self.pattern = is_op('nn.max_pool2d')(self.var)
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 2

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        
        original = post
        if self.cnt != self.target:
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')

            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32



In [51]:
params

{'_param_1': <tvm.nd.NDArray shape=(64, 3, 7, 7), cpu(0)>
 array([[[[ 5.76548587e-07,  5.64649895e-07,  6.48708181e-07, ...,
            6.83585881e-07,  7.50719039e-07,  7.35598348e-07],
          [ 7.98321423e-07,  9.42328882e-07,  8.98080771e-07, ...,
            8.56955864e-07,  8.45314844e-07,  7.00684268e-07],
          [ 6.81469601e-07,  8.55436724e-07,  7.91369700e-07, ...,
            6.94070934e-07,  6.97331416e-07,  6.54580788e-07],
          ...,
          [ 5.16474984e-07,  8.71320708e-07,  8.59098691e-07, ...,
            3.21061805e-07,  3.47937714e-07,  5.61184606e-07],
          [ 7.13678332e-07,  7.47119884e-07,  6.84571376e-07, ...,
            4.48654902e-07,  6.64658614e-07,  7.49582284e-07],
          [ 8.90952322e-07,  7.71111559e-07,  6.19053594e-07, ...,
            7.38091217e-07,  7.99400141e-07,  7.90432637e-07]],
 
         [[ 5.45430112e-07,  5.80063613e-07,  6.40928818e-07, ...,
            7.40630639e-07,  8.86499492e-07,  8.06069750e-07],
          [ 7.

In [4]:
class ReluCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        self.pattern = is_op('nn.relu')(self.var)
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 3

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        
        original = post
        if self.cnt != self.target:
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')

            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32



In [7]:
class TGICallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        bn_node = is_op('nn.batch_norm')(wildcard(), wildcard(), wildcard(), wildcard(), wildcard())
        tuple_get_item_node = is_tuple_get_item(bn_node, 0)
        self.pattern = tuple_get_item_node
        # self.pattern = is_op('nn.relu')(self.var)
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 3

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        print(self.cnt)
        original = post
        if self.cnt != self.target:
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')

            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32



In [ ]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        self.pattern = is_op('nn.max_pool2d')(self.var)
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 2

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        
        original = post
        if self.cnt != self.target:
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')

            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32



In [49]:
class LetCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        # self.pattern = is_let(wildcard(), wildcard(), wildcard())
        self.pattern = (wildcard())
        self.cnt = 0
        self.target = 2

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        
        original = post
        print(self.cnt)
        # print(post)

        if self.cnt != self.target:
            self.cnt += 1

            return post
        else:
            self.cnt += 1
            return post

            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')

            # cast_to_int8 = relay.cast(
            #     relay.clip(
            #         relay.round(
            #             relay.multiply(post, relay.const(16.0))
            #         ), 
            #         a_min=-127.0, a_max=127.0
            #     ),
            #     dtype="int8"
            # )
            # cast_to_float32 = relay.cast(
            #     relay.clip(
            #         relay.right_shift(
            #             relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
            #             relay.const(10)),
            #         a_min=-127.0, a_max=127.0), 
            #     dtype="float32"
            # )
            # return cast_to_float32



SyntaxError: invalid syntax (268454240.py, line 14)

In [8]:
out2 = rewrite(TGICallback(), mod['main'])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [9]:
print(out2)

fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_param_26

In [37]:
class BNCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        # self.pattern = ExprPattern(None)0
        self.pattern = is_op('')
        # self.pattern = TuplePattern(None)
        self.cnt = 0
        self.target = 3

    def callback(self, pre, post, node_map):
        original = post
        print(self.cnt)
        print(post)
        if self.cnt != self.target:
            self.cnt += 1
            return pre
        else:
            self.cnt += 1
            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')
    
            # cast_to_int8 = relay.cast(
            #     relay.clip(
            #         relay.round(
            #             relay.multiply(post, relay.const(16.0))
            #         ), 
            #         a_min=-127.0, a_max=127.0
            #     ),
            #     dtype="int8"
            # )
            # cast_to_float32 = relay.cast(
            #     relay.clip(
            #         relay.right_shift(
            #             relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
            #             relay.const(10)),
            #         a_min=-127.0, a_max=127.0), 
            #     dtype="float32"
            # )
            # return cast_to_float32
            return pre


In [38]:
out2 = rewrite(BNCallback(), mod['main'])


TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::Op (tvm::runtime::String)>::AssignTypedLambda<tvm::{lambda(tvm::runtime::String)#2}>(tvm::{lambda(tvm::runtime::String)#2}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)
  0: tvm::Op::Get(tvm::runtime::String const&)
  File "/home/j/tvm-slicer/include/tvm/src/ir/op.cc", line 46
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------
  Check failed: (reg != nullptr) is false: AttributeError: Operator  is not registe

In [45]:
mod['main'].body

Exception ignored in: <function ObjectBase.__del__ at 0x7f6f036d8c10>
Traceback (most recent call last):
  File "/home/j/tvm-slicer/include/tvm/python/tvm/_ffi/_ctypes/object.py", line 114, in __del__
    check_call(_LIB.TVMObjectFree(handle))
KeyboardInterrupt: 


In [21]:
out = rewrite(ReluCallback(), mod['main'])


In [22]:
out

fn (%input_1: Tensor[(1, 3, 224, 224), float32], %v_param_1: Tensor[(64, 3, 7, 7), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_19: Tensor[(256, 64, 1, 1), float32], %v_param_20: Tensor[(256), float32], %v_param_23: Tensor[(256), float32], %v_param_24: Tensor[(256), float32], %v_param_25: Tensor[(256), float32], %v_param_26: Tensor[(256), float32], %v_param_7: Tensor[(64, 64, 1, 1), float32], %v_param_8: Tensor[(64), float32], %v_param_9: Tensor[(64), float32], %v_param_10: Tensor[(64), float32], %v_param_11: Tensor[(64), float32], %v_param_12: Tensor[(64), float32], %v_param_13: Tensor[(64, 64, 3, 3), float32], %v_param_14: Tensor[(64), float32], %v_param_15: Tensor[(64), float32], %v_param_16: Tensor[(64), float32], %v_param_17: Tensor[(64), float32], %v_param_18: Tensor[(64), float32], %v_param_21: Tensor[(256, 64, 1, 1), float32], %v_p

In [10]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(out2, target, params=params)


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [11]:
show_graph(lib['get_graph_json'](), "unet_get_tuple_3")

In [25]:
lib.export_library("./resnet_3_q.so")

In [34]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        # self.var = wildcard()
        # self.var2 = wildcard()
        # self.pool_size = is_constant()
        # self.strides = is_constant()
        # self.padding = is_constant()
        # self.padding = [1, 1, 1, 1]
        # self.pattern = self.var
        self.pattern = is_op("nn.conv2d")(wildcard(), wildcard())
        self.cnt = 0
        self.target = 5

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        # print(node_map)
        # print(var)
        original = post
        # return pre
        if self.cnt != self.target:
            print(self.cnt)
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            # return relay.annotation.stop_fusion(post)
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32



In [35]:
out = rewrite(JUNOCallback(), mod['main'])


0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
16
17
18


In [23]:
print(out)

fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_param_26

In [36]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(out, target, params=params)

show_graph(lib['get_graph_json'](), "unet_3")

In [12]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(out, target, params=params)

show_graph(lib['get_graph_json'](), "quant_0")

The type inference pass was unable to infer a type for this expression.
This usually occurs when an operator call is under constrained in some way, check other reported errors for hints of what may of happened.
The type inference pass was unable to infer a type for this expression.
This usually occurs when an operator call is under constrained in some way, check other reported errors for hints of what may of happened.
The type inference pass was unable to infer a type for this expression.
This usually occurs when an operator call is under constrained in some way, check other reported errors for hints of what may of happened.
The type inference pass was unable to infer a type for this expression.
This usually occurs when an operator call is under constrained in some way, check other reported errors for hints of what may of happened.
The type inference pass was unable to infer a type for this expression.
This usually occurs when an operator call is under constrained in some way, check ot

DiagnosticError: Traceback (most recent call last):
  11: TVMFuncCall
  10: _ZNSt17_Function_handlerIFvN3tvm
  9: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  8: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, std::unordered_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, tvm::runtime::NDArray, std::hash<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, std::equal_to<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, std::allocator<std::pair<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const, tvm::runtime::NDArray> > > const&, tvm::runtime::String)
  7: tvm:

In [51]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(mod, target, params=params)
    # lib = relay.build(out, target, params=params)

show_graph(lib['get_graph_json'](), "unet_0")


In [ ]:
path = "../../tvm-slicer/src/model/unet_cuda_front_512_3_29.so"

In [ ]:
front_lib = tvm.runtime.load_module(path)

In [ ]:
print(front_lib['get_graph_json']())

{"nodes": [{"op": "null", "name": "input_0", "inputs": []}, {"op": "null", "name": "p0", "inputs": []}, {"op": "null", "name": "p1", "inputs": []}, {"op": "null", "name": "p2", "inputs": []}, {"op": "tvm_op", "name": "tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu", "attrs": {"num_outputs": "1", "num_inputs": "4", "flatten_data": "0", "func_name": "tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu", "out_layout": "", "kernel_layout": "OIHW", "data_layout": "NCHW", "hash": "3dbb7c2534747700"}, "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0]]}, {"op": "null", "name": "p3", "inputs": []}, {"op": "null", "name": "p4", "inputs": []}, {"op": "null", "name": "p5", "inputs": []}, {"op": "null", "name": "p6", "inputs": []}, {"op": "tvm_op", "name": "tvmgen_default_fused_nn_conv2d_add_multiply_add", "attrs": {"num_outputs": "1", "num_inputs": "5", "flatten_data": "0", "func_name": "tvmgen_default_fused_nn_conv2d_add_multiply_add", "out_layout": "", "kernel_layout": "OIHW", "data_l

In [ ]:
front_model = graph_executor.GraphModule(front_lib['default'](tvm.cuda()))

In [3]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("unet", 512))


2022-04-11 10:33:36.816309: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-11 10:33:36.816371: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-11 10:33:36.853566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 10:33:36.853856: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-11 10:33:36.853871: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-11 10:33:36.855403: I tensorflow/stream_executor/platfor

In [4]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("unet", 512))

input_data = np.random.normal(0,1,(1,512,512,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

In [68]:
print(mod)

def @main(%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_pa

In [81]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(), wildcard(), wildcard(), wildcard())
        self.pattern = (wildcard(), wildcard())
        self.cnt = 0
        self.target = 7

    def callback(self, pre, post, node_map):
        # var = node_map[self.var]
        print("check")
        original = post
        if self.cnt != self.target:
            self.cnt += 1
            return post
        else:
            self.cnt += 1

            # return relay.op.cast( 
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0))
            #     ,'float32')


            # return relay.op.cast( 
            #     relay.qnn.op.dequantize(
            #         relay.qnn.op.quantize(post, output_scale=relay.const(128), output_zero_point=relay.const(0)),
            #         input_scale=relay.const(128), input_zero_point=relay.const(0) 
            #     ),'float32')

            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32

In [59]:
x = relay.var('x', shape=(1, 3, 224, 224))
w = relay.var('w', shape=(3, 3, 3, 3))
b = relay.var('b', shape=(3, ))
conv2d = relay.op.nn.conv2d(x, w)
out = relay.op.nn.bias_add(conv2d, b)
func = relay.Function([x, w, b], out)
mod1 = tvm.IRModule.from_expr(func)


In [60]:
func

fn (%x: Tensor[(1, 3, 224, 224), float32], %w: Tensor[(3, 3, 3, 3), float32], %b: Tensor[(3), float32]) {
  %0 = nn.conv2d(%x, %w, padding=[0, 0, 0, 0]);
  nn.bias_add(%0, %b)
}

In [87]:
mod1[0]

#[version = "0.0.5"]
def @main(%x: Tensor[(1, 3, 224, 224), float32], %w: Tensor[(3, 3, 3, 3), float32], %b: Tensor[(3), float32]) {
  %0 = nn.conv2d(%x, %w, padding=[0, 0, 0, 0]);
  nn.bias_add(%0, %b)
}

In [88]:
func

fn (%x: Tensor[(1, 3, 224, 224), float32], %w: Tensor[(3, 3, 3, 3), float32], %b: Tensor[(3), float32]) {
  %0 = nn.conv2d(%x, %w, padding=[0, 0, 0, 0]);
  nn.bias_add(%0, %b)
}

In [82]:
out = rewrite(JUNOCallback(), mod['main'])


ValueError: don't know how to convert type <class 'function'> to object

In [80]:
print(out)

fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_param_26

In [67]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

In [ ]:
print(mod.astext())

#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3

In [ ]:
from SlicingMachine import TVMSlicer


In [ ]:
graph_json_raw = lib['get_graph_json']()
tvm_slicer = TVMSlicer(graph_json_raw)


In [ ]:
graph_json_front_info = tvm_slicer.slice_graph(0, 29)
graph_json_back_info = tvm_slicer.slice_graph(29, 111)


In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build_graph(mod, target=target, target_host=None, params=params, mod_name="default", graph_config=json.dumps(graph_json_front_info))
            

In [23]:
out = rewrite(JUNOCallback(), mod['main'])

pass
